In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install xmltodict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from urllib.request import urlopen
from urllib.request import Request 
from urllib.parse import urlencode, quote_plus

import urllib.request as ul
import xmltodict
import json
import sys
import io
import pandas as pd
import requests

### 연월일 분리위한 데이터 프레임 만들기

In [4]:
start = pd.to_datetime('20200829')
end = pd.to_datetime('20220701') 
df = pd.DataFrame(columns=['date'])
df['date'] = pd.date_range(start,end,freq='D')

df['Year'] = df['date'].dt.year 
df['Month'] = df['date'].dt.month 
df['Day'] = df['date'].dt.day

df

,date,Year,Month,Day
0,2020-08-29,2020,8,29
1,2020-08-30,2020,8,30
2,2020-08-31,2020,8,31
3,2020-09-01,2020,9,1
4,2020-09-02,2020,9,2
...,...,...,...,...
667,2022-06-27,2022,6,27
668,2022-06-28,2022,6,28
669,2022-06-29,2022,6,29
670,2022-06-30,2022,6,30


In [5]:
### 월,일의 한자리(ex. 1,2,3...) 앞에 0붙여야함
def convert_month(df):
    for i in range(len(df)):
        month = df['Month'].iloc[i]
        if int(month) < 10:
            month = "0"+str(month)
            df['Month'].iloc[i] = month
            
def convert_day(df):
    for i in range(len(df)):
        day = df['Day'].iloc[i]
        if int(day) < 10:
            day = "0"+str(day)
            df['Day'].iloc[i] = day        

In [6]:
convert_month(df)
convert_day(df)
df

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,date,Year,Month,Day
0,2020-08-29,2020,08,29
1,2020-08-30,2020,08,30
2,2020-08-31,2020,08,31
3,2020-09-01,2020,09,01
4,2020-09-02,2020,09,02
...,...,...,...,...
667,2022-06-27,2022,06,27
668,2022-06-28,2022,06,28
669,2022-06-29,2022,06,29
670,2022-06-30,2022,06,30


### API로 음력 받아오기

In [7]:
data = pd.DataFrame()


for i in range(len(df)):
    
    Year = df['Year'].iloc[i]
    Month = df['Month'].iloc[i]
    Day = df['Day'].iloc[i]
               
    url = 'http://apis.data.go.kr/B090041/openapi/service/LrsrCldInfoService/getLunCalInfo?serviceKey=MS8w0NUzG28tbYJgzzGzNKYispIbGKI3zFOsb8cV08xuh%2FCqBHlYn%2FfQcLySJxjsN0dvB93Ol%2BVQKftGRQG9ew%3D%3D&solYear='+str(Year)+'&solMonth='+str(Month)+'&solDay='+str(Day)

    requestd = Request(url)
    requestd.get_method = lambda: 'GET'
    response_body = urlopen(requestd).read()
    resp = xmltodict.parse(response_body)
    resp_dic = json.loads(json.dumps(resp))
    print(resp_dic)
    #print(i)

    data = data.append(resp_dic['response']['body']['items']['item'],ignore_index=True)

print("********end********")

{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': {'item': {'lunDay': '11', 'lunIljin': '갑진(甲辰)', 'lunLeapmonth': '평', 'lunMonth': '07', 'lunNday': '29', 'lunSecha': '경자(庚子)', 'lunWolgeon': '갑신(甲申)', 'lunYear': '2020', 'solDay': '29', 'solJd': '2459091', 'solLeapyear': '윤', 'solMonth': '08', 'solWeek': '토', 'solYear': '2020'}}, 'numOfRows': '10', 'pageNo': '1', 'totalCount': '1'}}}
{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': {'item': {'lunDay': '12', 'lunIljin': '을사(乙巳)', 'lunLeapmonth': '평', 'lunMonth': '07', 'lunNday': '29', 'lunSecha': '경자(庚子)', 'lunWolgeon': '갑신(甲申)', 'lunYear': '2020', 'solDay': '30', 'solJd': '2459092', 'solLeapyear': '윤', 'solMonth': '08', 'solWeek': '일', 'solYear': '2020'}}, 'numOfRows': '10', 'pageNo': '1', 'totalCount': '1'}}}
{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': {'item': {'lunDay': '13', 'lunIljin': '병오(

In [8]:
raw_api = data[['lunYear','lunMonth','lunDay']]
raw_api

,lunYear,lunMonth,lunDay
0,2020,07,11
1,2020,07,12
2,2020,07,13
3,2020,07,14
4,2020,07,15
...,...,...,...
667,2022,05,29
668,2022,05,30
669,2022,06,01
670,2022,06,02


### 만들어둔 데이터 프레임이랑 합치기

In [9]:
lun = pd.concat([df,raw_api],axis=1) 
lun

,date,Year,Month,Day,lunYear,lunMonth,lunDay
0,2020-08-29,2020,08,29,2020,07,11
1,2020-08-30,2020,08,30,2020,07,12
2,2020-08-31,2020,08,31,2020,07,13
3,2020-09-01,2020,09,01,2020,07,14
4,2020-09-02,2020,09,02,2020,07,15
...,...,...,...,...,...,...,...
667,2022-06-27,2022,06,27,2022,05,29
668,2022-06-28,2022,06,28,2022,05,30
669,2022-06-29,2022,06,29,2022,06,01
670,2022-06-30,2022,06,30,2022,06,02


In [10]:
# datetime 형태의 데이터프레임 생성
df_merge_date = pd.DataFrame(pd.date_range(start='2020-08-29', end='2022-07-02',freq = 'h',name='datetime', closed='left'))
df_merge_date['datetime'] = df_merge_date['datetime'].astype(str).str[:16]

In [11]:
#음력데이터와 합치기
df_merge_date['date']= df_merge_date['datetime'].str[:10] 
df_merge_date.info()
df_merge_date

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16128 entries, 0 to 16127
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   datetime  16128 non-null  object
 1   date      16128 non-null  object
dtypes: object(2)
memory usage: 252.1+ KB


,datetime,date
0,2020-08-29 00:00,2020-08-29
1,2020-08-29 01:00,2020-08-29
2,2020-08-29 02:00,2020-08-29
3,2020-08-29 03:00,2020-08-29
4,2020-08-29 04:00,2020-08-29
...,...,...
16123,2022-07-01 19:00,2022-07-01
16124,2022-07-01 20:00,2022-07-01
16125,2022-07-01 21:00,2022-07-01
16126,2022-07-01 22:00,2022-07-01


In [12]:
df_merge_date['datetime'] = pd.to_datetime(df_merge_date['datetime'])
df_merge_date['date'] = pd.to_datetime(df_merge_date['date'])
df_merge_date.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16128 entries, 0 to 16127
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   datetime  16128 non-null  datetime64[ns]
 1   date      16128 non-null  datetime64[ns]
dtypes: datetime64[ns](2)
memory usage: 252.1 KB


In [13]:
lun.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 672 entries, 0 to 671
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      672 non-null    datetime64[ns]
 1   Year      672 non-null    int64         
 2   Month     672 non-null    object        
 3   Day       672 non-null    object        
 4   lunYear   672 non-null    object        
 5   lunMonth  672 non-null    object        
 6   lunDay    672 non-null    object        
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 36.9+ KB


In [14]:
merged_lunar = pd.merge(df_merge_date,lun, how='inner', on='date')
merged_lunar = merged_lunar.sort_values('datetime').reset_index(drop=True)
merged_lunar['hour'] = merged_lunar['datetime'].astype(str).str[11:13].astype(int)
merged_lunar[['Month','Day','lunYear','lunMonth','lunDay']] = merged_lunar[['Month','Day','lunYear','lunMonth','lunDay']].astype(int)
merged_lunar = merged_lunar[['datetime','Year','Month','Day','hour','lunYear','lunMonth','lunDay']]
merged_lunar.columns = ['datetime','year','month','day','hour','lunYear','lunMonth','lunDay']
merged_lunar

,datetime,year,month,day,hour,lunYear,lunMonth,lunDay
0,2020-08-29 00:00:00,2020,8,29,0,2020,7,11
1,2020-08-29 01:00:00,2020,8,29,1,2020,7,11
2,2020-08-29 02:00:00,2020,8,29,2,2020,7,11
3,2020-08-29 03:00:00,2020,8,29,3,2020,7,11
4,2020-08-29 04:00:00,2020,8,29,4,2020,7,11
...,...,...,...,...,...,...,...,...
16123,2022-07-01 19:00:00,2022,7,1,19,2022,6,3
16124,2022-07-01 20:00:00,2022,7,1,20,2022,6,3
16125,2022-07-01 21:00:00,2022,7,1,21,2022,6,3
16126,2022-07-01 22:00:00,2022,7,1,22,2022,6,3


# 풍력 데이터 합치기

In [15]:
wind = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AIfactory_에너지 인공지능 경진대회/new_data/wind_forecast_f_sun.csv')
wind['datetime'] = pd.to_datetime(wind['datetime'])
wind.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16128 entries, 0 to 16127
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   datetime           16128 non-null  datetime64[ns]
 1   temperature        16128 non-null  float64       
 2   precipitationform  16128 non-null  float64       
 3   precipitationprob  16128 non-null  float64       
 4   humidity           16128 non-null  float64       
 5   windspeed          16128 non-null  float64       
 6   winddirection      16128 non-null  float64       
 7   cloud              16128 non-null  float64       
 8   precipitation      16128 non-null  float64       
 9   snow               16128 non-null  float64       
 10  sunrise            16128 non-null  int64         
 11  sunset             16128 non-null  int64         
 12  sunshine           16128 non-null  int64         
dtypes: datetime64[ns](1), float64(9), int64(3)
memory usage: 1.6 

In [16]:
wind_merged_lunar = pd.merge(merged_lunar,wind, how='inner', on='datetime')
wind_merged_lunar = wind_merged_lunar.sort_values('datetime').reset_index(drop=True)
wind_merged_lunar

,datetime,year,month,day,hour,lunYear,lunMonth,lunDay,temperature,precipitationform,precipitationprob,humidity,windspeed,winddirection,cloud,precipitation,snow,sunrise,sunset,sunshine
0,2020-08-29 00:00:00,2020,8,29,0,2020,7,11,26.0,0.0,30.0,95.0,1.400000,126.000000,4.0,0.0,0.0,6,19,13
1,2020-08-29 01:00:00,2020,8,29,1,2020,7,11,26.0,0.0,30.0,95.0,1.333333,125.666667,4.0,0.0,0.0,6,19,13
2,2020-08-29 02:00:00,2020,8,29,2,2020,7,11,26.0,0.0,30.0,95.0,1.266667,125.333333,4.0,0.0,0.0,6,19,13
3,2020-08-29 03:00:00,2020,8,29,3,2020,7,11,26.0,0.0,30.0,95.0,1.200000,125.000000,4.0,0.0,0.0,6,19,13
4,2020-08-29 04:00:00,2020,8,29,4,2020,7,11,26.0,0.0,30.0,95.0,1.266667,123.333333,4.0,0.0,0.0,6,19,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16123,2022-07-01 19:00:00,2022,7,1,19,2022,6,3,27.0,0.0,0.0,70.0,1.800000,142.000000,1.0,0.0,0.0,5,19,14
16124,2022-07-01 20:00:00,2022,7,1,20,2022,6,3,26.0,0.0,0.0,80.0,1.500000,140.000000,1.0,0.0,0.0,5,19,14
16125,2022-07-01 21:00:00,2022,7,1,21,2022,6,3,25.0,0.0,0.0,85.0,1.500000,133.000000,1.0,0.0,0.0,5,19,14
16126,2022-07-01 22:00:00,2022,7,1,22,2022,6,3,24.0,0.0,0.0,90.0,1.400000,126.000000,1.0,0.0,0.0,5,19,14


In [17]:
wind_merged_lunar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16128 entries, 0 to 16127
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   datetime           16128 non-null  datetime64[ns]
 1   year               16128 non-null  int64         
 2   month              16128 non-null  int64         
 3   day                16128 non-null  int64         
 4   hour               16128 non-null  int64         
 5   lunYear            16128 non-null  int64         
 6   lunMonth           16128 non-null  int64         
 7   lunDay             16128 non-null  int64         
 8   temperature        16128 non-null  float64       
 9   precipitationform  16128 non-null  float64       
 10  precipitationprob  16128 non-null  float64       
 11  humidity           16128 non-null  float64       
 12  windspeed          16128 non-null  float64       
 13  winddirection      16128 non-null  float64       
 14  cloud 

In [18]:
wind_merged_lunar.to_csv("/content/drive/MyDrive/Colab Notebooks/AIfactory_에너지 인공지능 경진대회/new_data/wind_forecast_f_sun_lun.csv",index=False)

# 태양광 데이터 합치기

In [19]:
solar = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AIfactory_에너지 인공지능 경진대회/new_data/solar_forecast_f_sun.csv')
solar['datetime'] = pd.to_datetime(solar['datetime'])
solar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16128 entries, 0 to 16127
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   datetime           16128 non-null  datetime64[ns]
 1   temperature        16128 non-null  float64       
 2   precipitationform  16128 non-null  float64       
 3   precipitationprob  16128 non-null  float64       
 4   humidity           16128 non-null  float64       
 5   windspeed          16128 non-null  float64       
 6   winddirection      16128 non-null  float64       
 7   cloud              16128 non-null  float64       
 8   precipitation      16128 non-null  float64       
 9   snow               16128 non-null  float64       
 10  sunrise            16128 non-null  int64         
 11  sunset             16128 non-null  int64         
 12  sunshine           16128 non-null  int64         
dtypes: datetime64[ns](1), float64(9), int64(3)
memory usage: 1.6 

In [20]:
solar_merged_lunar = pd.merge(merged_lunar,solar, how='inner', on='datetime')
solar_merged_lunar = solar_merged_lunar.sort_values('datetime').reset_index(drop=True)
solar_merged_lunar

,datetime,year,month,day,hour,lunYear,lunMonth,lunDay,temperature,precipitationform,precipitationprob,humidity,windspeed,winddirection,cloud,precipitation,snow,sunrise,sunset,sunshine
0,2020-08-29 00:00:00,2020,8,29,0,2020,7,11,26.0,0.0,20.000000,90.000000,0.9,174.0,3.000000,0.0,0.0,6,19,13
1,2020-08-29 01:00:00,2020,8,29,1,2020,7,11,26.0,0.0,23.333333,91.666667,0.9,176.0,3.333333,0.0,0.0,6,19,13
2,2020-08-29 02:00:00,2020,8,29,2,2020,7,11,26.0,0.0,26.666667,93.333333,0.9,178.0,3.666667,0.0,0.0,6,19,13
3,2020-08-29 03:00:00,2020,8,29,3,2020,7,11,26.0,0.0,30.000000,95.000000,0.9,180.0,4.000000,0.0,0.0,6,19,13
4,2020-08-29 04:00:00,2020,8,29,4,2020,7,11,26.0,0.0,30.000000,95.000000,0.9,138.0,4.000000,0.0,0.0,6,19,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16123,2022-07-01 19:00:00,2022,7,1,19,2022,6,3,26.0,0.0,0.000000,85.000000,2.4,262.0,1.000000,0.0,0.0,5,19,14
16124,2022-07-01 20:00:00,2022,7,1,20,2022,6,3,25.0,0.0,0.000000,90.000000,2.1,267.0,1.000000,0.0,0.0,5,19,14
16125,2022-07-01 21:00:00,2022,7,1,21,2022,6,3,24.0,0.0,0.000000,90.000000,1.5,257.0,1.000000,0.0,0.0,5,19,14
16126,2022-07-01 22:00:00,2022,7,1,22,2022,6,3,24.0,0.0,20.000000,90.000000,0.9,241.0,3.000000,0.0,0.0,5,19,14


In [21]:
solar_merged_lunar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16128 entries, 0 to 16127
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   datetime           16128 non-null  datetime64[ns]
 1   year               16128 non-null  int64         
 2   month              16128 non-null  int64         
 3   day                16128 non-null  int64         
 4   hour               16128 non-null  int64         
 5   lunYear            16128 non-null  int64         
 6   lunMonth           16128 non-null  int64         
 7   lunDay             16128 non-null  int64         
 8   temperature        16128 non-null  float64       
 9   precipitationform  16128 non-null  float64       
 10  precipitationprob  16128 non-null  float64       
 11  humidity           16128 non-null  float64       
 12  windspeed          16128 non-null  float64       
 13  winddirection      16128 non-null  float64       
 14  cloud 

In [22]:
solar_merged_lunar.to_csv("/content/drive/MyDrive/Colab Notebooks/AIfactory_에너지 인공지능 경진대회/new_data/solar_forecast_f_sun_lun.csv",index=False)